In [ ]:
# to import and execute as a Snowflake Notebook!

USE ROLE accountadmin;

In [ ]:
# must add "snowflake.core" to Packages!

from snowflake.core import Root
from snowflake.snowpark.context import get_active_session

session = get_active_session()
root = Root(session)
schema = root.databases["test"].schemas["public"]

### Image Repositories

In [ ]:
from snowflake.core.image_repository import ImageRepository

repos = schema.image_repositories
repo = repos.create(ImageRepository("tmp_repo"))

repo = repos["tmp_repo"]
print(repo.fetch().owner)
repo.drop()

for repo in repos.iter():
  print(repo.name)

### Compute Pools

In [ ]:
ALTER COMPUTE POOL cpu1 SUSPEND;
SHOW COMPUTE POOLS;

In [ ]:
pool = root.compute_pools["cpu1"]
print(pool.fetch().to_dict())
pool.suspend()

pools = root.compute_pools.iter()
for pool in pools:
    print(pool.name, pool.state, pool.min_nodes, pool.max_nodes, pool.instance_family)

In [ ]:

from snowflake.core.compute_pool import ComputePool

name = "tmp_pool"
pool_def = ComputePool(name=name,
    instance_family="CPU_X64_XS",
    min_nodes=1, max_nodes=1)
pool = root.compute_pools.create(pool_def)
snapshot = pool.fetch()
data = root.compute_pools.iter(like="tmp%")

pool = root.compute_pools[name].fetch()
pool.max_nodes = 3
pool_res = root.compute_pools[name].create_or_alter(pool)

pool = root.compute_pools[name].fetch()
print(pool.max_nodes)

pool = root.compute_pools[name]
pool.stop_all_services()
pool.suspend()
pool.delete()

### Services

In [ ]:
# must upload "service1_spec.yml" local file to existing @stage1

session.file.put("service1_spec.yml", "@stage1", auto_compress=False)

# could also create a staged file from inline string
repo_url = schema.image_repositories["repo"].fetch().repository_url
spec = f"""
    spec:
      containers:
      - name: job1c
        image: {repo_url}/job1
    """
from io import BytesIO
session.file.put_stream(
    BytesIO(bytes(spec, encoding='utf-8')),
    "@stage1/service1_spec_str.yml",
    auto_compress=False)

In [ ]:
USE ROLE sysadmin;

-- RM @stage1 pattern='.*.';
LIST @stage1;

In [ ]:
from snowflake.core.service import Service, ServiceSpec

# from spec in staged file
service_def = Service(name="tmp_service",
    min_instances=1, max_instances=2,
    compute_pool="cpu1",
    spec=ServiceSpec("@stage1/service1_spec.yml"))
service = schema.services.create(service_def)

service = schema.services["tmp_service"]
service.suspend()
service.resume()
status = service.get_service_status(10)
print(status)

service.delete();

In [ ]:
# from inline spec as string
service_def = Service(name="tmp_service",
    min_instances=1, max_instances=2,
    compute_pool="cpu1",
    spec=ServiceSpec(spec))
service = schema.services.create(service_def)

for service in schema.services.iter(like="%"):
    print(service.name)

service = schema.services["tmp_service"]
service.delete();

In [ ]:
SHOW SERVICES IN SCHEMA test.public;

In [ ]:
ALTER COMPUTE POOL cpu1 SUSPEND;
SHOW COMPUTE POOLS;